In [1]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import random


from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np
from sklearn import model_selection
import pandas as pd

In [9]:
train_data = np.genfromtxt('train.csv', delimiter=',')
train_data = train_data[1:, :]
test_data = np.genfromtxt('test.csv', delimiter=',')
test_data = test_data[1:, :]


x = train_data[:,:-1]
y = train_data[:, -1]
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.2)

print(train_data[np.where(train_data[:, -1]== 1)].shape)
print(train_data[np.where(train_data[:, -1]== 0)].shape)

def get_bootstrap(data, labels, n):
    n_samples = data.shape[0]
    bootstrap = []

    # by rows in the sample
    for i in range(n):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        # by columns in the sample 
        for j in range(n_samples):
            # shuffling index by random
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]

        bootstrap.append((b_data, b_labels))

    return bootstrap

def get_subsample(len_sample):
    # list of indixes from zero to len
    sample_indices = [i for i in range(len_sample)]
    # length of subsample as sqr root of the whole length
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    # shuffling the indices of the sample
    random.shuffle(sample_indices)

    for i in range(len_subsample):
        subsample.append(sample_indices.pop())

    return subsample

class Node:
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index # index of the feature comapred to threshold
        self.t = t # threshold value
        self.true_branch = true_branch # subsample in compliance with the condition in the node
        self.false_branch = false_branch # subsample not in compliance with the condition in the node

# base case node == leaf
class Leaf:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        # qaunt of the objects of various classes
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        prediction = max(classes, key=classes.get)

        return prediction

def gini(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2

    return impurity

# calc the quality 
def quality(left_labels, right_labels, current_gini):
    # samples elements in the left subtree
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

# splitting dataset in the node
def split(data, labels, index, t):

    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]

    return true_data, false_data, true_labels, false_labels

def find_best_split(data, labels):

    # min quant of objects in the node
    min_leaf = 1

    current_gini = gini(labels)

    best_quality, best_t, best_index = 0, None, None

    n_features = data.shape[1]

    # subsample size = sq root of the sample size
    subsample = get_subsample(n_features)

    for index in subsample:
        # getting only unique values of a feature
        t_values = np.unique([row[index] for row in data])

        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            # skipping splits with less than 5 objects within
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue

            current_quality = quality(true_labels, false_labels, current_gini)

            # getting threshold with the max increase in quality
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

def build_tree(data, labels):
    quality, t, index = find_best_split(data, labels)

    # base case if not quality increase
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # building true & false subtrees recursively
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    return Node(index, t, true_branch, false_branch)

# building random forest
def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)

    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))

    return forest

# classification function
def classify_object(obj, node):

    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

# prediction for the sample by one tree
def predict(data, tree):

    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)

    return classes

# prediction by trees' vote
def tree_vote(forest, data):

    # all prediction in a list
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))

    # list with predictions for the each of the objects
    predictions_per_object = list(zip(*predictions))

    # the preidction with the most of votes
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))

    return voted_predictions

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

n_trees = 10
my_forest_1 = random_forest(x_train, y_train, n_trees)

(0, 12)
(0, 12)


In [10]:
train_answers = tree_vote(my_forest_1, x_train)

In [11]:
test_answers = tree_vote(my_forest_1, x_test)

In [12]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(y_train, train_answers)
print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {train_accuracy:.3f}')

Точность случайного леса из 10 деревьев на обучающей выборке: 70.213
